<a href="https://colab.research.google.com/github/anshhhul/Automated-Crowd-Detection/blob/main/CrowdDetectionProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

The following commands to install the necessary libraries for computer vision, PyTorch (for the YOLO model), and image processing.

These commands will install:

(1) PyTorch and torchvision for loading and using the YOLO model.

(2) OpenCV for image manipulation.

(3) Pillow and matplotlib for image processing and displaying results.

In [ ]:
# Install PyTorch and YOLOv5 dependencies
!pip install torch torchvision torchaudio
!pip install opencv-python pillow matplotlib tqdm

The following to load YOLOv5’s small version (yolov5s), which is optimized for speed and suitable for quick testing.

In [ ]:
import torch

# Load YOLOv5 model (small version)
model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)
model.eval()  # Set the model to evaluation mode


/usr/local/lib/python3.10/dist-packages/torch/hub.py:330: UserWarning: You are about to download and run code from an untrusted repository. In a future release, this won't be allowed. To add the repository to your trusted list, change the command to {calling_fn}(..., trust_repo=False) and a command prompt will appear asking for an explicit confirmation of trust, or load(..., trust_repo=True), which will assume that the prompt is to be answered with 'yes'. You can also use load(..., trust_repo='check') which will only prompt for confirmation if the repo is not already trusted. This will eventually be the default behaviour
  warnings.warn(
Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to /root/.cache/torch/hub/master.zip


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


YOLOv5 🚀 2024-11-9 Python-3.10.12 torch-2.5.0+cu121 CPU

100%|██████████| 14.1M/14.1M [00:00<00:00, 126MB/s]

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 


AutoShape(
  (model): DetectMultiBackend(
    (model): DetectionModel(
      (model): Sequential(
        (0): Conv(
          (conv): Conv2d(3, 32, kernel_size=(6, 6), stride=(2, 2), padding=(2, 2))
          (act): SiLU(inplace=True)
        )
        (1): Conv(
          (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
          (act): SiLU(inplace=True)
        )
        (2): C3(
          (cv1): Conv(
            (conv): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1))
            (act): SiLU(inplace=True)
          )
          (cv2): Conv(
            (conv): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1))
            (act): SiLU(inplace=True)
          )
          (cv3): Conv(
            (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1))
            (act): SiLU(inplace=True)
          )
          (m): Sequential(
            (0): Bottleneck(
              (cv1): Conv(
                (conv): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1))
  

Here, we set the model to evaluation mode with model.eval(), which tells the model we are only doing inference, not training.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Specify the folder path in Google Drive containing your input images
folder_path = 'My Drive/crowd_images'

import os

# List all image files in the specified folder
image_paths = [os.path.join(folder_path, img) for img in os.listdir(folder_path) if img.endswith(('.jpg', '.png'))]

print("Found images:", image_paths)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


FileNotFoundError: [Errno 2] No such file or directory: 'My Drive/crowd_images'

YOLOv5 expects images to be in a certain format (640x640 pixels), so we need to preprocess our images before passing them to the model.



In [ ]:
import cv2

def preprocess_image(image_path):
    # Load the image
    img = cv2.imread(image_path)
    # Resize to YOLOv5's input size (640x640 by default)
    img_resized = cv2.resize(img, (640, 640))
    return img_resized


A function to count the number of people detected in each image and classify the crowd level.

Define Thresholds for Crowd Levels:

We’ll use simple thresholds to classify the crowd based on the number of people detected:

Low: 0–10 people

Moderate: 11–20 people

High: 21+ people

In [ ]:
def classify_crowd_level(num_people):
    # Define thresholds for crowd levels
    if num_people <= 10:
        return "Low"
    elif num_people <= 20:
        return "Moderate"
    else:
        return "High"

def detect_and_classify_crowd(image):
    # Run YOLOv5 on the image
    results = model(image)
    people_count = 0

    # Count detections for 'person' class
    for detection in results.xyxy[0]:  # bbox format: x1, y1, x2, y2, confidence, class
        if int(detection[-1]) == 0:  # Class 0 is 'person' in YOLO
            people_count += 1

    # Classify the crowd level
    crowd_level = classify_crowd_level(people_count)
    return people_count, crowd_level


Here:

detect_and_classify_crowd runs YOLO on the image and counts the number of detected people (class ID 0).

It then classifies the crowd level using classify_crowd_level.

In [ ]:
import matplotlib.pyplot as plt

for image_name in uploaded.keys():
    # Preprocess and load the image
    image = preprocess_image(image_name)

    # Detect and classify crowd
    people_count, crowd_level = detect_and_classify_crowd(image)

    # Display results
    print(f"Image: {image_name} - People detected: {people_count} - Crowd level: {crowd_level}")

    # Optional: Display the image with overlay
    plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    plt.title(f"People: {people_count} | Crowd Level: {crowd_level}")
    plt.axis('off')
    plt.show()


/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):


Image: DALL·E 2024-11-09 14.03.38 - A color image of the inside of a public bus, showing a moderate crowd of people. Passengers are standing and sitting, with some holding onto handrails.webp - People detected: 12 - Crowd level: Moderate


This code:

Processes each image, detects the number of people, and classifies the crowd level.

Displays the image with the crowd level and people count using matplotlib.

To save the crowd detection data (e.g., the number of people and crowd level) for each image, you can store this information in a CSV file.

In [ ]:
import cv2
import os
import csv
from google.colab import drive

# Step 1: Mount Google Drive
drive.mount('/content/drive')

# Step 2: Define paths for folders in Google Drive
base_folder = 'My Drive/crowd_detection_project'
input_folder = os.path.join(base_folder, 'input_images')  # Where you upload your images
processed_folder = os.path.join(base_folder, 'processed_images')
results_folder = os.path.join(base_folder, 'results')

# Create folders if they don’t exist
os.makedirs(input_folder, exist_ok=True)
os.makedirs(processed_folder, exist_ok=True)
os.makedirs(results_folder, exist_ok=True)

# Define CSV file path for storing detection results
csv_file_path = os.path.join(results_folder, 'crowd_detection_results.csv')

# Step 3: Get all image paths from the input folder
image_paths = [os.path.join(input_folder, img) for img in os.listdir(input_folder) if img.endswith(('.jpg', '.png'))]

# Step 4: Open CSV file and start writing results
with open(csv_file_path, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["Image Name", "People Count", "Crowd Level"])  # Write CSV header

    # Process each image in the input folder
    for image_path in image_paths:
        # Load each image
        image = cv2.imread(image_path)

        # Run detection and classification (replace detect_and_classify_crowd with your function)
        people_count, crowd_level = detect_and_classify_crowd(image)

        # Add overlay text with detection results
        cv2.putText(image, f"People count: {people_count} - Crowd level: {crowd_level}", (10, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)

        # Save processed image to processed folder
        output_path = os.path.join(processed_folder, f"processed_{os.path.basename(image_path)}")
        cv2.imwrite(output_path, image)
        print(f"Saved processed image to: {output_path}")

        # Write results to CSV
        writer.writerow([os.path.basename(image_path), people_count, crowd_level])
        print(f"Saved results to CSV for image: {os.path.basename(image_path)}")

print(f"All results saved to {csv_file_path}")
